In [ ]:
import json, os
import pandas as pd
import numpy as np
import scipy.stats as stats

from statsmodels.tsa.api import VAR
from datetime import timedelta, datetime

from aquabyte.data_access_utils import RDSAccessUtils

pd.set_option('display.max_rows', None)

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

In [ ]:
earliestDate = '2019-12-01'
allFields = [ 'female_avg', 'moving_avg' ]
rollingMADays = 5
halfRollingMADays = int((rollingMADays + 1)/ 2)
maxAR = 3

def getTrainingAndTestSeries(inputJSON):    
    penId = inputJSON['penId']
    date = datetime.strptime(inputJSON['date'], '%Y-%m-%d')

    query = """
        select date, female_avg, moving_avg
        from day_summaries a
        where a.pen_id = %i
        and a.date <= '%s'
        and a.date >= '%s';
    """ % (penId, date, earliestDate)

    day_summaries = rds_access_utils.extract_from_database(query)

    day_summaries.index = pd.to_datetime(day_summaries['date'])
    day_summaries = day_summaries.sort_index()

    day_summaries_rolling = day_summaries.rolling('%iD' % (rollingMADays, )).mean().shift(-24 * rollingMADays / 2, freq='h').resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
    day_summaries_rolling.ix[0:halfRollingMADays] = np.nan
    day_summaries_rolling = day_summaries_rolling.dropna()

    query = """
        select event_type, started_at, ended_at from event_logs 
        where pen_id = %i
        and started_at <= '%s'
        and ended_at >= '%s'
        and event_type = 'WELLBOAT_TREATMENT'
        order by started_at asc;
    """ % (penId, date, earliestDate)

    allEvents = rds_access_utils.extract_from_database(query)

    eligiblePeriods = []
    lastEndedAt = day_summaries_rolling.index[0]

    for index, event in allEvents.iterrows():
        if abs((lastEndedAt - event['started_at'].replace(tzinfo=None)).days) > 30:
            eligiblePeriods.append((lastEndedAt, event['started_at'].replace(tzinfo=None)))
            print('Adding period from %s to %s' % (lastEndedAt, event['started_at']))
        else:
            print('Cannot add period from %s to %s' % (lastEndedAt, event['started_at']))

        if event['ended_at'] is not None:
            if lastEndedAt is None:
                lastEndedAt = event['ended_at']
            lastEndedAt = max(lastEndedAt, event['ended_at'].replace(tzinfo=None))
        else:
            print('Treatment still in progress')

    lastDay = day_summaries_rolling.index[-1]

    if abs((lastEndedAt - lastDay).days) > 30:
        eligiblePeriods.append((lastEndedAt, lastDay))  
    else:
        print('Cannot add period from %s to %s' % (lastEndedAt, lastDay))

    lastEligiblePeriod = None

    if len(eligiblePeriods) > 0:
        lastEligiblePeriod = eligiblePeriods[-1]
    else:
        print('No eligible training periods')

    if abs((lastEndedAt - lastDay).days) < maxAR + 1:
        print('Not enough data to generate AR series')

    print('Using period from %s to %s to train' % (lastEligiblePeriod[0], lastEligiblePeriod[1]))

    goodTrainingPeriod = (day_summaries_rolling.index > lastEligiblePeriod[0]) & (day_summaries_rolling.index < lastEligiblePeriod[1].replace(minute=0, hour=0, second=0, microsecond=0))
    trainingSeries = day_summaries_rolling[goodTrainingPeriod]
    testSeries = day_summaries_rolling.ix[-(maxAR + 1):]
    
    return trainingSeries, testSeries

In [ ]:
def trainAndTestFutureLiceModel(inputJSON, trainingSeries, testSeries):
    forecastPeriodDays = inputJSON['forecastPeriodDays']
    
    exponentialFields = []

    trainingX = trainingSeries.copy()
    logTrainingX = np.log(trainingX)
    trainingDiffX = trainingX.diff().dropna()
    logTrainingDiffX = logTrainingX.diff().dropna()

    testX = testSeries.copy()
    logTestX = np.log(testX)
    testDiffX = testX.diff().dropna()
    logTestDiffX = logTestX.diff().dropna()

    #size = min(30, len(trainingDiffX))
    size = len(trainingDiffX)

    for field in allFields:
        k2, p = stats.normaltest(logTrainingDiffX[field][-size:])
        #print(field, p)
        if p > .05 and field == 'female_avg':
            print('Adding %s exponential: %0.2f' % (field, p))
            exponentialFields.append(field)
            trainingDiffX[field] = logTrainingDiffX[field]
            testX[field] = logTestX[field]
            testDiffX[field] = logTestDiffX[field]
        else:
            print('Keeping %s non-exponential: %0.2f' % (field, p))

    predictions = {}
    predictionsSeries = testSeries.copy()
    predictionsSeriesLower = testSeries.copy()
    predictionsSeriesUpper = testSeries.copy()

    model = VAR(trainingDiffX[-size:])
    model_fit = model.fit(maxlags = maxAR, ic = 'aic') #model.fit(AR)
    lag_order = model_fit.k_ar
    if lag_order == 0:
        model_fit = model.fit(1)

    output, output_lower, output_upper = model_fit.forecast_interval(testDiffX.values[-lag_order:], forecastPeriodDays + halfRollingMADays, 0.25)

    convertedField = {
        'female_avg': 'adultFemale',
        'moving_avg': 'mobile'
    }

    cov = model_fit.forecast_cov(forecastPeriodDays + halfRollingMADays)

    currentDate = testX.index[-1]
    currentValue = testX.ix[-1]

    for i in range(forecastPeriodDays + halfRollingMADays):
        predictionDate = currentDate + timedelta(days = i + 1)
        predictionDateString = predictionDate.strftime('%Y-%m-%d')
        datePrediction = currentValue + np.sum(output[:i], 0)
        datePredictionLower = currentValue + np.sum(output_lower[:i], 0)
        datePredictionUpper = currentValue + np.sum(output_upper[:i], 0)

        SE = np.sqrt(np.diagonal(np.sum(cov[:i,:,:], 0)))
        currentCOV = np.diagonal(cov[i,:,:])
        expSE = np.sqrt(np.sum((np.exp(currentCOV) - 1) * np.exp(2 * np.mean(trainingDiffX) + currentCOV), 0))

        if i >= halfRollingMADays:
            predictions[predictionDateString] = {}

        for field in allFields:
            datePredictionValue = datePrediction[field]
            datePredictionLowerValue = datePredictionLower[field]
            datePredictionUpperValue = datePredictionUpper[field]

            if i >= halfRollingMADays:
                predictions[predictionDateString][convertedField[field]] = {
                    'smartAvg': np.maximum(datePredictionValue, 0.0),
                    'smartAvgLowerThreshold': np.maximum(np.maximum(datePredictionValue - SE[1] * 1.96, datePredictionLowerValue), 0.0),
                    'smartAvgUpperThreshold': np.maximum(np.minimum(datePredictionValue + SE[1] * 1.96, datePredictionUpperValue), 0.0)
                }

            predictionsSeries.loc[predictionDate] = np.maximum(datePrediction, 0.0)
            predictionsSeriesLower.loc[predictionDate] = np.maximum(np.maximum(datePrediction - SE * 1.96, datePredictionLower), 0.0)
            predictionsSeriesUpper.loc[predictionDate] = np.maximum(np.minimum(datePrediction + SE * 1.96, datePredictionUpper), 0.0)


        datePredictionExp = np.maximum(np.exp(datePrediction), 0.0)
        datePredictionLowerExp = np.maximum(np.maximum(np.exp(datePrediction) - expSE * 1.96, np.exp(datePredictionLower)), 0.0)
        datePredictionUpperExp = np.maximum(np.minimum(np.exp(datePrediction) + expSE * 1.96, np.exp(datePredictionUpper)), 0.0)

        for field in exponentialFields:
            datePredictionValue = datePredictionExp[field]
            datePredictionLowerValue = datePredictionLowerExp[field]
            datePredictionUpperValue = datePredictionUpperExp[field]

            if i >= halfRollingMADays:
                predictions[predictionDateString][convertedField[field]] = {
                    'smartAvg': datePredictionValue,
                    'smartAvgLowerThreshold': datePredictionLowerValue,
                    'smartAvgUpperThreshold': datePredictionUpperValue
                }

            predictionsSeries.loc[predictionDate, field] = datePredictionValue
            predictionsSeriesLower.loc[predictionDate, field] = datePredictionLowerValue
            predictionsSeriesUpper.loc[predictionDate, field] = datePredictionUpperValue

    return predictions, predictionsSeries, predictionsSeriesLower, predictionsSeriesUpper

In [ ]:
def futureLiceLevels(inputJSON):
    trainingSeries, testSeries = getTrainingAndTestSeries(inputJSON)
    
    predictions, _ , _ , _ = trainAndTestFutureLiceModel(inputJSON, trainingSeries, testSeries)
    
    return predictions

In [ ]:
inputJSON = {
  "penId": 56,
  "date": "2020-04-08",
  "forecastPeriodDays": 14
}

predictions = futureLiceLevels(inputJSON)

import pprint
pp = pprint.PrettyPrinter(depth=6)

pp.pprint(predictions)

In [ ]:
def futureLiceLevelsPlot(inputJSON):
    trainingSeries, testSeries = getTrainingAndTestSeries(inputJSON)
    
    _ , predictionsSeries, predictionsSeriesLower, predictionsSeriesUpper = trainAndTestFutureLiceModel(inputJSON, trainingSeries, testSeries)
    
    return testSeries, predictionsSeries, predictionsSeriesLower, predictionsSeriesUpper

In [ ]:
inputJSON = {
  "penId": 56,
  "date": "2020-04-08",
  "forecastPeriodDays": 14
}

testSeries, predictionsSeries, predictionsSeriesLower, predictionsSeriesUpper = futureLiceLevelsPlot(inputJSON)

import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

fig.set_size_inches(15, 20)

ax[0].plot(testSeries.index, testSeries['female_avg'], color = 'black', linewidth = 5, label = 'Rolling')
ax[0].plot(predictionsSeries.index, predictionsSeries['female_avg'], color = 'purple', linewidth = 2, label = 'Daily Predictions')
ax[0].fill_between(predictionsSeries.index, predictionsSeriesLower['female_avg'], predictionsSeriesUpper['female_avg'], color='b', alpha=.1)
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Adult Female Count')
#ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#ax[0].axhline(0)

ax[1].plot(testSeries.index, testSeries['moving_avg'], color = 'black', linewidth = 5, label = 'Rolling')
ax[1].plot(predictionsSeries.index, predictionsSeries['moving_avg'], color = 'purple', linewidth = 2, label = 'Daily Predictions')
ax[1].fill_between(predictionsSeries.index, predictionsSeriesLower['moving_avg'], predictionsSeriesUpper['moving_avg'], color='b', alpha=.1)
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Moving Count')